In [ ]:
# -*- coding: utf-8 -*-

import pandas as pd
import numpy as np
from scipy.signal import savgol_filter
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Define a function to normalize data
def normalize(df, cols):
    return (df[cols] - df[cols].min()) / (df[cols].max() - df[cols].min())

# Resampling and interpolation to handle varying frequencies
def resample_data(df, target_frequency=30):
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
    df = df.set_index('timestamp').resample(f'{int(1000 / target_frequency)}ms').mean()
    df = df.interpolate(method='linear').reset_index()
    return df

# Integrate acceleration to obtain velocity
def integrate_acceleration_to_velocity(df, sampling_rate):
    velocity = np.cumsum(df[['x', 'y', 'z']].values, axis=0) * (1 / sampling_rate)
    return pd.DataFrame(velocity, columns=['vx', 'vy', 'vz'])

# Integrate velocity to estimate distance
def integrate_velocity_to_distance(velocity_df, sampling_rate):
    distance = np.cumsum(np.sqrt(velocity_df['vx']**2 + velocity_df['vy']**2 + velocity_df['vz']**2)) * (1 / sampling_rate)
    return distance

# Compute features for a given segment
def compute_features(segment):
    features = {
        'mean_x': segment['x'].mean(),
        'std_x': segment['x'].std(),
        'integral_x': np.trapz(segment['x']),
        # Repeat for y and z
    }
    return features

# Evaluate models
def evaluate_model(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    return mae, rmse, r2

# Simulate longer trajectories by concatenating 5-meter segments
def create_longer_trajectories(df, target_distance=50):
    repeat_count = target_distance // 5
    longer_trajectory = pd.concat([df] * repeat_count).reset_index(drop=True)
    cumulative_distance = np.cumsum([5] * len(longer_trajectory))
    return longer_trajectory, cumulative_distance

# Main code for processing datasets and model training
motion_types = ['curve_slow'] #, 'curve_fast', 'straight_slow', 'straight_fast']
all_results = []

# Data Augmentation
def augment_data(df, jitter_std=0.01, scaling_factor_range=(0.9, 1.1), time_warp_factor_range=(0.8, 1.2)):
    augmented_dfs = []
    labels = []
    # Original data (unchanged)
    augmented_dfs.append(df.copy())


    labels.append("original")

    # Jittering: Add random noise to simulate sensor variation
    jittered_df = df.copy()
    jittered_df[['x', 'y', 'z']] += np.random.normal(0, jitter_std, df[['x', 'y', 'z']].shape)
    augmented_dfs.append(jittered_df)
    labels.append("jittering")

    # Scaling: Adjust amplitude of the sensor readings
    scaling_factor = np.random.uniform(*scaling_factor_range)
    scaled_df = df.copy()
    scaled_df[['x', 'y', 'z']] *= scaling_factor
    augmented_dfs.append(scaled_df)
    labels.append("scaling")

    # Time Warping: Stretch or compress the time axis
    time_warp_factor = np.random.uniform(*time_warp_factor_range)
    warped_df = df.copy()
    warped_indices = np.round(np.linspace(0, len(df) - 1, int(len(df) * time_warp_factor))).astype(int)
    warped_indices = np.clip(warped_indices, 0, len(df) - 1)
    warped_df = df.iloc[warped_indices].reset_index(drop=True)
    augmented_dfs.append(warped_df)
    labels.append("time_warping")

    return augmented_dfs,labels

for motion in motion_types:
    print(f"Processing {motion} data...")

    # Load datasets
    accelerometer = pd.read_csv(f"/content/sample_data/{motion}_Accelerometer.csv")
    gyroscope = pd.read_csv(f"/content/sample_data/{motion}_Gyroscope.csv")

   # print("Original Accelerometer Data:")
   # print(accelerometer.head(), "\n")

    # Use 'seconds_elapsed' as timestamp
    if 'seconds_elapsed' in accelerometer.columns:
        accelerometer.rename(columns={'seconds_elapsed': 'timestamp'}, inplace=True)
    else:
        raise ValueError("Expected 'seconds_elapsed' column in Accelerometer data!")

    # Normalize and smooth accelerometer data
    accelerometer[['x', 'y', 'z']] = normalize(accelerometer, ['x', 'y', 'z'])

    #print("Normalized Accelerometer Data:")
    #print(accelerometer[['timestamp', 'x', 'y', 'z']].head(), "\n")

    accelerometer_smoothed = savgol_filter(accelerometer[['x', 'y', 'z']], window_length=5, polyorder=2, axis=0)
    accelerometer[['x', 'y', 'z']] = accelerometer_smoothed

    #print("Smoothed Accelerometer Data:")
    #print(accelerometer[['timestamp', 'x', 'y', 'z']].head(), "\n")

    # Resample data
    accelerometer_resampled = resample_data(accelerometer)

   # print("Resampled Accelerometer Data:")
   # print(accelerometer_resampled.head(), "\n")

    #sampling_rate = 30  # Hz after resampling

    # Step 1: Integrate acceleration to obtain velocity
    #velocity_df = integrate_acceleration_to_velocity(accelerometer_resampled[['x', 'y', 'z']], sampling_rate)

    # Step 2: Integrate velocity to estimate distance
    #distance_estimated = integrate_velocity_to_distance(velocity_df, sampling_rate)

    # Step 3: Define ground truth distances using 5-meter increments
    #ground_truth_distance = np.cumsum([5] * (len(distance_estimated) // (sampling_rate * 5)))  # Reset every 5 meters


      # Apply Data Augmentation
    #augmented_datasets = augment_data(accelerometer_resampled)
    augmented_datasets, augmentation_labels = augment_data(accelerometer_resampled)
   # print(f"Augmented Data for {motion} (Sample Output for Jittering):")
   # print(augmented_datasets[1].head(), "\n")  # Jittered data sample

  #  print(f"Augmented Data for {motion} (Sample Output for Scaling):")
  #  print(augmented_datasets[2].head(), "\n")  # Scaled data sample

  #  print(f"Augmented Data for {motion} (Sample Output for Time Warping):")
   # print(augmented_datasets[3].head(), "\n")  # Time-warped data sample

    # Process each augmented dataset
    for  augmented_df,augmentation_type in zip(augmented_datasets, augmentation_labels):
        # Extract features, estimate distance, and prepare data for training
        sampling_rate = 30  # Hz after resampling
        velocity_df = integrate_acceleration_to_velocity(augmented_df[['x', 'y', 'z']], sampling_rate)
        distance_estimated = integrate_velocity_to_distance(velocity_df, sampling_rate)

        # Create cumulative distance target
        cumulative_distance = np.cumsum([5] * (len(distance_estimated) // (sampling_rate * 5)))

        # Define X (features) and y (target)
        X = augmented_df[['x', 'y', 'z']]
        y = cumulative_distance

        # Split data into train and test sets
        #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


        # Step 4: Create synthetic longer trajectories
        longer_trajectory, cumulative_distance = create_longer_trajectories(accelerometer_resampled, target_distance=50)

        # Define X (features) and y (target)
        X = longer_trajectory[['x', 'y', 'z']]
        y = cumulative_distance

        # Split data into train and test sets
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Train Random Forest Regressor
        rfr = RandomForestRegressor(n_estimators=100, random_state=42)
        rfr.fit(X_train, y_train)
        y_pred_rfr = rfr.predict(X_test)

        # Train Support Vector Regressor
        svr = SVR(kernel='rbf', C=10, epsilon=0.1)
        svr.fit(X_train, y_train)
        y_pred_svr = svr.predict(X_test)

        # Evaluate models
        rfr_metrics = evaluate_model(y_test, y_pred_rfr)
        svr_metrics = evaluate_model(y_test, y_pred_svr)
        all_results.append({
             "motion": motion,
             "augmentation": augmentation_type,
             "model": "Random Forest",
             "MAE": rfr_metrics[0],
             "RMSE": rfr_metrics[1],
             "R2": rfr_metrics[2]
         })
        all_results.append({
             "motion": motion,
             "augmentation": augmentation_type,
             "model": "SVR",
             "MAE": svr_metrics[0],
             "RMSE": svr_metrics[1],
             "R2": svr_metrics[2]
         })

# Display results
results_df = pd.DataFrame(all_results)
print(results_df)

# Find the best Random Forest result (based on lowest RMSE)
best_rfr = results_df[results_df['model'] == 'Random Forest'].sort_values(by='RMSE', ascending=True).iloc[0]

# Find the best SVR result (based on lowest RMSE)
best_svr = results_df[results_df['model'] == 'SVR'].sort_values(by='RMSE', ascending=True).iloc[0]

# Print the best results
print("\nBest Random Forest Result:")
print(best_rfr)

print("\nBest SVR Result:")
print(best_svr)



Processing curve_slow data...
       motion  augmentation          model           MAE          RMSE  \
0  curve_slow      original  Random Forest  47826.093672  55414.041889   
1  curve_slow      original            SVR  42742.801793  49356.322766   
2  curve_slow     jittering  Random Forest  47826.093672  55414.041889   
3  curve_slow     jittering            SVR  42742.801793  49356.322766   
4  curve_slow       scaling  Random Forest  47826.093672  55414.041889   
5  curve_slow       scaling            SVR  42742.801793  49356.322766   
6  curve_slow  time_warping  Random Forest  47826.093672  55414.041889   
7  curve_slow  time_warping            SVR  42742.801793  49356.322766   

         R2  
0 -0.260705  
1 -0.000136  
2 -0.260705  
3 -0.000136  
4 -0.260705  
5 -0.000136  
6 -0.260705  
7 -0.000136  

Best Random Forest Result:
motion             curve_slow
augmentation         original
model           Random Forest
MAE              47826.093672
RMSE             55414.041889